一番最初に実行するコード

病床数の穴埋めのためのプログラム
2020/4/23~

In [13]:
import pandas as pd
from datetime import datetime as dt

In [14]:
start_day = dt(2020,7,30,0,0)
end_day = dt(2021,9,18,0,0)
date_type = '%Y-%m-%d'

filename1 = 'severe-bed-usage.csv'
filename2 = 'mild-moderate-bed-usage.csv'
filename3 = 'bed-usage.csv'
save_filename = 'bed_data.csv'

In [15]:
import openpyxl

#データ関連関数
def get_value_list(t_2d):
    return([[cell.value for cell in row] for row in t_2d])

def get_list_2d(sheet, start_row, end_row, start_col, end_col):
    return get_value_list(sheet.iter_rows(min_row=start_row,
                                          max_row=end_row,
                                          min_col=start_col,
                                         max_col=end_col))

def write_list_2d(sheet, l_2d, start_row, start_col):
    for y, row in enumerate(l_2d):
        for x, cell in enumerate(row):
            sheet.cell(row=start_row + y,
                       column=start_col + x,
                       value=l_2d[y][x])

In [16]:
#データの読み込み
#parse_datesで日にちの型として認識できるようにする
#index_colで日にちから認識するように指定
df1 = pd.read_csv('./data/'+filename1,index_col=[0], encoding='shift-jis', parse_dates=['日付'])

#データに欠損が存在したときに、補完するプログラム
# axis=0で時系列の方向に補完することできます。
df1 = df1.interpolate(axis=0)

In [17]:
df1

,重症病床確保数,重症入院患者数,重症病床使用率
日付,,,
2020-04-23,127,58,45.7
2020-04-24,155,59,38.1
2020-04-25,155,61,39.4
2020-04-26,155,59,38.1
2020-04-27,170,59,34.7
...,...,...,...
2022-03-03,629,213,33.9
2022-03-04,632,201,31.8
2022-03-05,632,194,30.7


In [18]:
'''df1.reset_index(inplace=True)
df1'''

'df1.reset_index(inplace=True)\ndf1'

In [19]:
#データの読み込み
df2 = pd.read_csv('./data/'+filename2,index_col=[0], encoding='shift-jis', parse_dates=True)

'''# axis=0で時系列の方向に補完することできます。
df2 = df2.interpolate(axis=0)'''
df2

,軽症中等症病床確保数,軽症中等症入院患者数,軽症中等症病床使用率
日付,,,
2020-04-23,656,539,82.2
2020-04-24,717,520,72.5
2020-04-25,717,520,72.5
2020-04-26,717,502,70.0
2020-04-27,808,510,63.1
...,...,...,...
2022-03-03,3419,2654,77.6
2022-03-04,3414,2662,78.0
2022-03-05,3408,2659,78.0


In [20]:
#データの読み込み
df3 = pd.read_csv('./data/'+filename3,index_col=[0], encoding='shift-jis', parse_dates=True)
df3

,病床確保数,入院患者数,病床使用率
日付,,,
2020-04-23,783,597,76.2
2020-04-24,872,579,66.4
2020-04-25,872,581,66.6
2020-04-26,872,561,64.3
2020-04-27,978,569,58.2
...,...,...,...
2022-03-03,4048,2867,70.8
2022-03-04,4046,2863,70.8
2022-03-05,4040,2853,70.6


In [21]:
#結合
#結合１、２、共通項目
df = pd.merge(df1, df2, on='日付')
df = pd.merge(df, df3["病床使用率"], on='日付')
df

,重症病床確保数,重症入院患者数,重症病床使用率,軽症中等症病床確保数,軽症中等症入院患者数,軽症中等症病床使用率,病床使用率
日付,,,,,,,
2020-04-23,127,58,45.7,656,539,82.2,76.2
2020-04-24,155,59,38.1,717,520,72.5,66.4
2020-04-25,155,61,39.4,717,520,72.5,66.6
2020-04-26,155,59,38.1,717,502,70.0,64.3
2020-04-27,170,59,34.7,808,510,63.1,58.2
...,...,...,...,...,...,...,...
2022-03-03,629,213,33.9,3419,2654,77.6,70.8
2022-03-04,632,201,31.8,3414,2662,78.0,70.8
2022-03-05,632,194,30.7,3408,2659,78.0,70.6


In [22]:
print(len(df.index))
drop_index = []
for index in df.index:
    if index < start_day or index > end_day:
        drop_index.append(index)
df = df.drop(index=drop_index)
df

684


,重症病床確保数,重症入院患者数,重症病床使用率,軽症中等症病床確保数,軽症中等症入院患者数,軽症中等症病床使用率,病床使用率
日付,,,,,,,
2020-07-30,188,18,9.6,1069,245,22.9,20.9
2020-07-31,188,19,10.1,1069,275,25.7,23.4
2020-08-01,188,20,10.6,1069,288,26.9,24.5
2020-08-02,188,23,12.2,1069,341,31.9,29.0
2020-08-03,188,27,14.4,1069,346,32.4,29.7
...,...,...,...,...,...,...,...
2021-09-14,605,256,42.3,2730,1761,64.5,60.5
2021-09-15,605,238,39.3,2758,1650,59.8,56.1
2021-09-16,605,224,37.0,2758,1589,57.6,53.9


In [23]:
#データの丸め
df = df.round()
df

,重症病床確保数,重症入院患者数,重症病床使用率,軽症中等症病床確保数,軽症中等症入院患者数,軽症中等症病床使用率,病床使用率
日付,,,,,,,
2020-07-30,188,18,10.0,1069,245,23.0,21.0
2020-07-31,188,19,10.0,1069,275,26.0,23.0
2020-08-01,188,20,11.0,1069,288,27.0,24.0
2020-08-02,188,23,12.0,1069,341,32.0,29.0
2020-08-03,188,27,14.0,1069,346,32.0,30.0
...,...,...,...,...,...,...,...
2021-09-14,605,256,42.0,2730,1761,64.0,60.0
2021-09-15,605,238,39.0,2758,1650,60.0,56.0
2021-09-16,605,224,37.0,2758,1589,58.0,54.0


In [24]:
df.to_csv('./data/'+save_filename, encoding='shift-jis')